In [35]:
import torch
import torchvision

# The complete list of built-in transforms from pytorch can be see [here](https://pytorch.org/vision/stable/transforms.html).

## On Images
* CenterCrop, Grayscale, Pad, RandomAffine
* RandomCrop, RandomHorizontalFlip, RandomRotation
* Resize, Scale

## On Tensors
* LinearTransformation, Normalize, RandomErasing

## Conversion
* ToPILImage : from tensor or ndarray
* ToTensor : from nummpy.ndarray or PILImage

## Generic
* Use Lambda

## Custom
* Write own class

## Compose multiple Transforms
### composed = transforms.Compose( [ Rescale(256), RandomCrop(224) ] )
### torchvision.transform.ReScalse(256)
### torchvision.transforms.ToTensor()

In [36]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np


class WineDataset(Dataset):  
    def __init__(self,*,transform = None):
        # data loading
        xy = np.loadtxt('./dataset/wine.csv',delimiter=',',dtype=np.float32,skiprows=1)
        # note we do not convert tensor here.
        self.x = xy[:,1:]
        self.y = xy[:,[0]]
        self.n_samples = xy.shape[0]
        # putting zero in square bracket makes the dimension of y into two. y.shape => [no_of_samples,1]
        
        # if we did y = xy[:,0] => this will give one dimension
        # y.shape => (no_of_samples,)
        self.transform = transform
    
    def __getitem__(self,index):
        # indexing dataset
        sample =  self.x[index],self.y[index] 
        if self.transform:
            sample =self.transform(sample)          
        return sample

    def __len__(self):
        # len(dataset)
        return self.n_samples
    


# To Write own custom class, we will write them as callable classes instead of simple functions so that parameters of the transform need not be passed everytime it’s called. For this, <span style = 'color:cyan'>we just need to implement __call__ method and if required, __init__ method.</span> 

ToTensor transform class does not required init method as it does not need input parameter while initiating the class.

MulitplicationTransform needs init method as we need an argument of the multiplcation factor.

In [37]:
class ToTensor: 
    # write classes where the instances behave like functions and can be called like a function.
    def __call__(self,sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)


class MultiplicationTransform:
    def __init__(self,factor):
        self.factor = factor
    
    def __call__(self,sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs,targets

In [38]:
composed = torchvision.transforms.Compose([ToTensor(),MultiplicationTransform(2)])
dataset = WineDataset(transform=composed)
composed_first_data = dataset[0]
print(composed_first_data)
print(f'type of the composed_first_data is {type(composed_first_data[0])}')

(tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03]), tensor([1.]))
type of the composed_first_data is <class 'torch.Tensor'>


In [39]:
dataset = WineDataset(transform=None)
first_data= dataset[0]
print(first_data[0])
print(f'type of the first_data is {type(first_data[0])}')

[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
type of the first_data is <class 'numpy.ndarray'>


In [44]:
print('Composed data is 2 times the original Data?\n')
composed_first_data[0][0].item() == 2*first_data[0][0].item()

Composed data is 2 times the original Data?



True